In [1]:
import os
import docx
import pandas as pd
from docx.shared import Pt
from docx.enum.section import WD_ORIENT,WD_SECTION
from decimal import Decimal

In [2]:
meta_exp_file = "/data5/deepro/starrseq/papers/results/5_compare_expression_ko_vs_wt/data/meta_exp.csv"

In [3]:
meta_exp_df = pd.read_csv(meta_exp_file)

In [4]:
highly_active_tfs = [
    "ATF4", "ZFX", "CEBPB", "JUN", "TP53", "CREB1", "ELK4",
    "JUND", "JUNB", "FOS", "FOSB", "FOSL1", "FOSL2", "ATF3", 
    ]

KOs = ["ATF2", "CTCF", "FOXA1", "LEF1", "SCRT1", "TCF7L2"]

In [5]:
table_df = meta_exp_df.loc[meta_exp_df.gene_name.isin(highly_active_tfs), ["gene_name"] + [f"{ko}_{info}" for ko in KOs for info in ["log2FoldChange", "padj"]]].sort_values("gene_name")

In [6]:
table_df.columns = [" ".join(c.split("_")) for c in table_df.columns]

In [7]:
# Initialise the Word document
doc = docx.Document()
# Document settings
style = doc.styles['Normal']
font = style.font
font.name = 'Times New Roman'
font.size = Pt(12)
heading_style = doc.styles['Heading 1']
heading_font = heading_style.font
heading_font.name = "Times New Roman"
heading_font.size = Pt(16)

def change_orientation():
    current_section = doc.sections[-1]
    new_width, new_height = current_section.page_height, current_section.page_width
    new_section = doc.add_section(WD_SECTION.NEW_PAGE)
    new_section.orientation = WD_ORIENT.LANDSCAPE
    new_section.page_width = new_width
    new_section.page_height = new_height
    return new_section

change_orientation()
doc.add_heading("Expression of TFs that bind to highly active fragments", 1)
t = doc.add_table(rows=1, cols=table_df.shape[1])
t.style = 'TableGrid'
# Add the column headings
for j in range(table_df.shape[1]):
    t.cell(0, j).text = table_df.columns[j]
# Add the body of the data frame
for i in range(table_df.shape[0]):
    row = t.add_row()
    for j in range(table_df.shape[1]):
        cell = table_df.iat[i, j]
        if not type(cell) == str:
            cell = f"{cell:.2E}"
        row.cells[j].text = cell

doc.save("../data/tables/highly_active_fragments_TF.docx")

/data5/deepro/miniconda3/envs/starrseq/lib/python3.9/site-packages/docx/styles/styles.py:139: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)
